In [1]:
import sys
# Uncomment if tornet isn't installed in your environment or in your path already
#sys.path.append('../')  

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras

from tornet.data.tf.loader import create_tf_dataset 
from tornet.data.constants import ALL_VARIABLES

In [ ]:
# keras accepts most data loaders (tensorflow, torch).
# A pure keras data loader, with necessary preprocessing steps for the cnn baseline, is provided
from tornet.data.keras.loader import KerasDataLoader
data_root = "C:/Users/mjhig/tornet_2013"
ds = KerasDataLoader(data_root=data_root,
                     data_type='train',
                     years=[2013, 2014, 2015, 2016, 2017, 2018],
                     batch_size = 128, 
                     workers = 4,
                     use_multiprocessing = True)

In [7]:
# Build a test set
ds_test = KerasDataLoader(data_root=data_root,
                         data_type='test',
                         years=[2018],
                         batch_size = 128, 
                         workers = 4,
                         use_multiprocessing = True)

In [ ]:
X_first, y_first = ds[0]  
unique_classes = np.unique(y_first)

In [23]:
x,y=ds[0]
y.shape

(128,)

In [25]:
x={ k: x[k] for k in ALL_VARIABLES }
x.keys()

dict_keys(['DBZ', 'VEL', 'KDP', 'RHOHV', 'ZDR', 'WIDTH'])

In [33]:
X_batch, y_batch = ds[0]
X_batch = np.stack([X_batch[k] for k in ALL_VARIABLES], axis=-1)
X_batch = X_batch.reshape(X_batch.shape[0], -1)
X_batch.shape
#classifier.partial_fit(X_batch, y_batch, classes=unique_classes)

(128, 345600)

In [42]:
len(ds)

28

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer
from 

SGDClassifier
classifier = SGDClassifier()
X_first, y_first = ds[0]  
unique_classes = np.unique(y_first)
for i in range(1, len(ds)):  # Start from batch 1
    X_batch, y_batch = ds[i]
    X_batch = np.stack([X_batch[k] for k in ALL_VARIABLES], axis=-1)
    X_batch = X_batch.reshape(X_batch.shape[0], -1)
    classifier.fit(X_batch, y_batch)
    print(f"Batch {i} done")

KeyboardInterrupt: 

In [44]:

import numpy as np
from sklearn.metrics import accuracy_score

# Store true labels and predictions
y_true = []
y_pred = []

# Iterate through test dataset batches
for i in range(len(ds_test)):  
    X_batch, y_batch = ds_test[i]  # Load batch

    # Extract and stack only relevant features
    X_batch = np.stack([X_batch[k] for k in ALL_VARIABLES], axis=-1)

    # Flatten spatial dimensions (same as training)
    X_batch = X_batch.reshape(X_batch.shape[0], -1)

    # Handle NaN values (replace with zero)
    X_batch = np.nan_to_num(X_batch, nan=0.0)

    # Predict batch labels
    preds = classifier.predict(X_batch)

    # Store true labels and predictions
    y_true.extend(y_batch)
    y_pred.extend(preds)

    print(f"Batch {i} prediction done")

# Convert to NumPy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")




Batch 0 prediction done
Batch 1 prediction done
Batch 2 prediction done
Batch 3 prediction done
Batch 4 prediction done
Batch 5 prediction done
Batch 6 prediction done


KeyboardInterrupt: 

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score, accuracy_score
from collections import Counter

# Store cumulative true labels and predictions
y_true_cumulative = []
y_pred_cumulative = []

# Iterate through test dataset batches
for i in range(len(ds_test)):  
    X_batch, y_batch = ds_test[i]  # Load batch

    # Extract and stack only relevant features
    X_batch = np.stack([X_batch[k] for k in ALL_VARIABLES], axis=-1)

    # Flatten spatial dimensions (same as training)
    X_batch = X_batch.reshape(X_batch.shape[0], -1)

    # Handle NaN values (replace with zero)
    X_batch = np.nan_to_num(X_batch, nan=0.0)

    # Predict batch labels
    preds = classifier.predict(X_batch)

    # Store true labels and predictions for the batch
    y_true_batch = np.array(y_batch)
    y_pred_batch = np.array(preds)

    # If classifier supports probability outputs, use them for AUC calculations
    if hasattr(classifier, "predict_proba"):
        y_pred_proba_batch = classifier.predict_proba(X_batch)[:, 1]  # Probability of class 1
    else:
        y_pred_proba_batch = y_pred_batch  # Use binary predictions if proba isn't available

    # Compute batch-wise metrics
    accuracy_batch = accuracy_score(y_true_batch, y_pred_batch)
    precision_batch = precision_score(y_true_batch, y_pred_batch, zero_division=0)
    recall_batch = recall_score(y_true_batch, y_pred_batch, zero_division=0)
    auc_batch = roc_auc_score(y_true_batch, y_pred_proba_batch) if len(np.unique(y_true_batch)) > 1 else float('nan')
    pr_auc_batch = average_precision_score(y_true_batch, y_pred_proba_batch) if len(np.unique(y_true_batch)) > 1 else float('nan')

    # Print batch metrics
    print(f"Batch {i} Metrics:")
    print(f" - Class Balance: {Counter(y_true_batch)}")
    print(f" - Accuracy: {accuracy_batch:.4f}")
    print(f" - Precision: {precision_batch:.4f}")
    print(f" - Recall: {recall_batch:.4f}")
    print(f" - AUC-ROC: {auc_batch:.4f}")
    print(f" - PR AUC: {pr_auc_batch:.4f}")
    print("-" * 40)

    # Update cumulative storage
    y_true_cumulative.extend(y_true_batch)
    y_pred_cumulative.extend(y_pred_batch)

# Convert cumulative lists to NumPy arrays
y_true_cumulative = np.array(y_true_cumulative)
y_pred_cumulative = np.array(y_pred_cumulative)

# Compute cumulative metrics
accuracy_cum = accuracy_score(y_true_cumulative, y_pred_cumulative)
precision_cum = precision_score(y_true_cumulative, y_pred_cumulative, zero_division=0)
recall_cum = recall_score(y_true_cumulative, y_pred_cumulative, zero_division=0)
auc_cum = roc_auc_score(y_true_cumulative, y_pred_cumulative) if len(np.unique(y_true_cumulative)) > 1 else float('nan')
pr_auc_cum = average_precision_score(y_true_cumulative, y_pred_cumulative) if len(np.unique(y_true_cumulative)) > 1 else float('nan')

# Print cumulative metrics
print("Cumulative Metrics:")
print(f" - Accuracy: {accuracy_cum:.4f}")
print(f" - Precision: {precision_cum:.4f}")
print(f" - Recall: {recall_cum:.4f}")
print(f" - PR AUC: {pr_auc_cum:.4f}")
print("=" * 40)



Batch 0 Metrics:
 - Class Balance: Counter({0: 122, 1: 6})
 - Accuracy: 0.7422
 - Precision: 0.0000
 - Recall: 0.0000
 - AUC-ROC: 0.3893
 - PR AUC: 0.0469
----------------------------------------
Batch 1 Metrics:
 - Class Balance: Counter({0: 116, 1: 12})
 - Accuracy: 0.7422
 - Precision: 0.1379
 - Recall: 0.3333
 - AUC-ROC: 0.5589
 - PR AUC: 0.1085
----------------------------------------
Batch 2 Metrics:
 - Class Balance: Counter({0: 123, 1: 5})
 - Accuracy: 0.7891
 - Precision: 0.0769
 - Recall: 0.4000
 - AUC-ROC: 0.6024
 - PR AUC: 0.0542
----------------------------------------
Batch 3 Metrics:
 - Class Balance: Counter({0: 123, 1: 5})
 - Accuracy: 0.7422
 - Precision: 0.0333
 - Recall: 0.2000
 - AUC-ROC: 0.4821
 - PR AUC: 0.0379
----------------------------------------
Batch 4 Metrics:
 - Class Balance: Counter({0: 118, 1: 10})
 - Accuracy: 0.7734
 - Precision: 0.1481
 - Recall: 0.4000
 - AUC-ROC: 0.6025
 - PR AUC: 0.1061
----------------------------------------


KeyboardInterrupt: 